# Step 5: Clustering of Retrosynthesis Routes

In retrosynthetic analysis, multiple synthetic routes can be generated for a single target molecule. Clustering these routes helps to group together pathways that share similar high-level strategies, making it easier to identify fundamentally different approaches and prioritize the most promising ones. This step enables chemists to focus on unique synthetic ideas rather than redundant variations.

This tutorial demonstrates how to cluster the Retrosynthesis Routes by unique strategic bond patterns.
At this stage you should have a set of solved retrosynthetic routes (either from running the planning step here or from loading previously saved results).

In [ ]:
#@title SynPlanner Installation
%pip install -q "synplanner>=1.2.0"


## 1. Set up input and output data locations

In [ ]:
import os
import pickle
import shutil
from pathlib import Path
from synplan.utils.loading import download_selected_files

# download only necessary SynPlanner assets
assets = [
    ("building_blocks", "building_blocks_em_sa_ln.smi"),
    ("uspto", "uspto_reaction_rules.pickle"),
    ("uspto/weights", "ranking_policy_network.ckpt"),
]

data_folder = download_selected_files(
    files_to_get=assets,
    save_to="synplan_data",
    extract_zips=True,
)

# input data
# You can use the input data:
# 1. Curated USPTO data, extracted reaction rules, pretrained policy network and building blocks from downloaded data
# 2. Use curated USPTO data, extracted reaction rules, pretrained policy network prepared with tutorials

USE_DOWNLOAD_DATA = True # is False if you want to use the data and models from previous tutorial steps
if USE_DOWNLOAD_DATA:
    # input data
    ranking_policy_network = data_folder.joinpath("uspto/weights/ranking_policy_network.ckpt").resolve(strict=True)
    reaction_rules_path = data_folder.joinpath("uspto/uspto_reaction_rules.pickle")
    # planning reslts folder
    planning_results_folder = Path("planning_with_downloaded_data").resolve()
    planning_results_folder.mkdir(exist_ok=True)
    clustering_results_folder = Path("clustering_with_downloaded_data").resolve()
    clustering_results_folder.mkdir(exist_ok=True)
else:
    # input data
    tutorial_results_folder = Path("tutorial_results").resolve()
    ranking_policy_network = tutorial_results_folder.joinpath("ranking_policy_network/policy_network.ckpt").resolve(strict=True)
    reaction_rules_path = tutorial_results_folder.joinpath("uspto_reaction_rules.pickle")
    # planning reslts folder
    results_folder = Path("planning_with_tutorial_data").resolve()
    results_folder.mkdir(exist_ok=True)

# use your custom building blocks if needed
building_blocks_path = data_folder.joinpath("building_blocks/building_blocks_em_sa_ln.smi")

In [ ]:
from synplan.utils.loading import load_building_blocks

building_blocks = load_building_blocks(building_blocks_path, standardize=True)

In [ ]:
from synplan.utils.loading import load_reaction_rules

reaction_rules = load_reaction_rules(reaction_rules_path)

In [ ]:
from synplan.mcts.expansion import PolicyNetworkFunction
from synplan.utils.config import PolicyNetworkConfig

policy_config = PolicyNetworkConfig(weights_path=ranking_policy_network) 
policy_network = PolicyNetworkFunction(policy_config=policy_config)

In [ ]:
from synplan.utils.config import TreeConfig

tree_config = TreeConfig(
    search_strategy="expansion_first",
    evaluation_type="rollout",
    max_iterations=300,
    max_time=120,
    max_depth=9,
    min_mol_size=1,
    init_node_value=0.5,
    ucb_type="uct",
    c_ucb=0.1,
)

In [ ]:
from synplan.chem.utils import mol_from_smiles

# let's take capivasertib used as anti-cancer medication for the treatment 
# of breast cancer and approved by FDA in 2023
example_smiles = "NC1(C(=O)N[C@@H](CCO)c2ccc(Cl)cc2)CCN(c2nc[nH]c3nccc2-3)CC1"

target_molecule = mol_from_smiles(example_smiles, clean2d=True, standardize=True, clean_stereo=True)

In [ ]:
from synplan.mcts.tree import Tree

tree = Tree(
    target=target_molecule,
    config=tree_config,
    reaction_rules=reaction_rules,
    building_blocks=building_blocks,
    expansion_function=policy_network,
    # you can also specify evaluation_function=ValueNetwork(...), by default it is None
)

In [ ]:
tree_solved = False
for solved, node_id in tree:
    if solved:
        tree_solved = True
tree

## 1.2. Download retrosynthetic planning results

**Input Data Structure**

The clustering workflow accepts routes in several formats:
 * CSV: Tabular format with route information.
 * JSON: Structured format with detailed route data.
 * Serialized tree: Pickled object containing the full search tree.

Each format can be loaded into a routes_dict, which is a Python dictionary mapping route IDs to route data. Choose the loading method that matches your available data

In [ ]:
from synplan.chem.reaction_routes.io import *

From CSV file

In [ ]:
csv_path = clustering_results_folder.joinpath("routes_1_1.csv")
routes_dict_1 = read_routes_csv(csv_path)

From JSON file

In [ ]:
json_path = clustering_results_folder.joinpath("routes_1_1.json")

routes_dict_2 = read_routes_json(file_path=json_path, to_dict=True)

From serialized tree

In [ ]:
# serialized tree should be named as tree_{mol_id}_{config}
#      were the mol_id is the id of the target molecule
#      and the config is the string representation of the tree config
ser_tree_path = clustering_results_folder.joinpath('forest')
mol_id = 1
config = '1'
tree_2 = TreeWrapper.load_tree_from_id(mol_id, config, ser_tree_path)

In [ ]:
routes_json = make_json(routes_dict_2)

## 2. Extract RouteCGR and ReducedRouteCGR

Here we convert each multi-step synthetic route into a single graph representation that captures all bond-breaking and bond-forming events:

- **Condensed Graph of Reaction (CGR):** Overlay reactants and products of each reaction step using atom mappings.  
- **RouteCGR:** Merge the per-step CGRs into one unified graph that preserves connectivity changes across the entire pathway.  
- **ReducedRouteCGR:** Filter the RouteCGR to retain only the “strategic bonds” (those involving atoms present in the target), removing extraneous leaving-group and reagent details.  
- **Visualization:** Render the resulting CGR to confirm that only core bond transformations remain.

In [ ]:
from synplan.chem.reaction_routes.route_cgr import *
from synplan.chem.reaction_routes.clustering import *
from synplan.chem.reaction_routes.visualisation import cgr_display
from IPython.display import display, HTML, SVG
from synplan.utils.visualisation import get_route_svg, get_route_svg_from_json, render_svg

The function compose_all_route_cgrs can process synthetic routes extracted either from a reaction tree or from CSV/JSON files containing a routes_dict.

In [ ]:
all_route_cgrs = compose_all_route_cgrs(tree_2)
# or
all_route_cgrs = compose_all_route_cgrs(routes_dict_2)

In [ ]:
i = 1
for route_id, route_cgr in all_route_cgrs.items():
    print(route_id)
    cgr_prods = [route_cgr.substructure(c) for c in route_cgr.connected_components]
    target_cgr = cgr_prods[0]
    display(SVG(cgr_display(target_cgr)))
    display(SVG(get_route_svg_from_json(routes_json, route_id)))
    # or 
    display(SVG(get_route_svg(tree, route_id))) # Currently the pathway from the serialized tree can not be depicted
    if i >= 3:
        break
    i += 1


In [ ]:
all_reduced_route_cgrs = compose_all_reduced_route_cgrs(all_route_cgrs)

In [ ]:
i = 1
for route_id, route_cgr in all_reduced_route_cgrs.items():
    print(route_id)
    cgr_prods = [route_cgr.substructure(c) for c in route_cgr.connected_components]
    target_cgr = cgr_prods[0]
    display(SVG(cgr_display(target_cgr)))
    display(SVG(get_route_svg_from_json(routes_json, route_id)))
    # or 
    display(SVG(get_route_svg(tree, route_id)))
    if i >= 3:
        break
    i += 1

## 3. Clustering

**How Does Clustering Work?**

* Routes are grouped based on their ReducedRouteCGR representations, which capture the strategic bond changes.
* The use_strat parameter controls whether clustering is based on structural signatures, ensuring that routes differing only in atom mappings are grouped together.
* The output is a dictionary mapping cluster indices to lists of route IDs.

In [ ]:
# use_strat: if True, clustering will use the CGRContainer’s structural signature
#            to ensure that routes which are chemically identical but differ only
#            in their atom mappings are grouped together instead of split apart
clusters = cluster_routes(all_reduced_route_cgrs, use_strat=False)

In [ ]:
clusters

### Clustering report file:

This guide walks you through how to assemble an interactive HTML report that displays, for each cluster of synthetic routes:
- The **target molecule** (SMILES string)
- The **cluster index**
- The **cluster size** (number of distinct pathways in the cluster)
- A **RouteCGR** graphic highlighting the common strategic bond transformations
- A gallery (“roll”) of each individual route showing:
    * Number of steps
    * (Optional) route score
    * Image of the route
    * Reactions SMILES for each transformation

In [ ]:
cluster_index = '5.1'
if cluster_index in clusters.keys():
    # display(HTML(routes_clustering_report(tree, clusters, cluster_index,
    #                      all_reduced_route_cgrs)))
    # or
    display(HTML(routes_clustering_report(routes_json, clusters, cluster_index,
                         all_reduced_route_cgrs)))
else:
    print(f"Cluster {cluster_index} not found in the clustering results.")


## 4. Subclustering

**Why Subcluster?**

* Subclustering reveals finer variations within each main cluster by abstracting away non-strategic details (e.g., leaving groups, protective groups).

- **Abstraction:** Replace all non-core groups in the original RouteCGR (e.g., leaving groups, protective groups) with a generic placeholder (e.g., “X”).  
- **Pseudo-Reaction Generation:** Export each abstracted graph as a Markush-style pseudo-SMILES or simplified representation.
- **Leaving group collection:** Identify, label and collect all leaving groups from the original RouteCGR.

* This step helps to further differentiate routes that share the same core strategy but differ in peripheral modifications

    Example: Secondary amine formation strategies

    1. Coupling a haloalkane with a primary amine
    2. Reductive amination (formally a two-step process)
    
    In the initial clustering layer, these routes are grouped together because they share the same core approach. However, subclustering allows us to differentiate between them based on their specific modifications.

* The output includes Markush-style pseudo-SMILES and a collection of leaving groups for each subgroup.

In [ ]:
all_subclusters = subcluster_all_clusters(clusters, all_reduced_route_cgrs, all_route_cgrs)

In [ ]:
cluster_index = '3.1'
subcluster_num = 1

if subcluster_num in all_subclusters[cluster_index].keys():
    subgroup = all_subclusters[cluster_index][subcluster_num]
    display(HTML(routes_subclustering_report(tree, subgroup, cluster_index, subcluster_num, all_reduced_route_cgrs, aam=False)))
else:
    print(f"Cluster {cluster_index} not found in the subclustering results.")

### Post-processing the subclustering result (Under development):

1. Remove ubiquitous leaving‑groups
 - Within any subgroup, if every pathway contains the same leaving‑group (or set of leaving‑groups), delete that X column.
 - Then reinstate the original parent compound from which that X was derived.
 - This reduces the table’s width and focuses on variable elements.

2. Merge duplicate routes
 - If two or more routes share an identical set of leaving‑groups, collapse them into a single entry.
 - In the first column (route ID), list all merged route IDs (one per row).

3. Unify symmetric pseudo‑reactions
 - When subgroups exhibit the same Markus‑style pseudo‑reaction SMILES (same bond changes) but differ only in atom‑atom mapping, merge them into one.
 - Because building‑block symmetry makes these mappings chemically equivalent, combining them avoids artificial duplication.

In [ ]:
if len(subgroup['nodes_data']) != 1:
    new_subgroup = post_process_subgroup(subgroup)
    display(HTML(routes_subclustering_report(tree, new_subgroup, cluster_index, subcluster_num, all_reduced_route_cgrs, if_lg_group=True)))

## References & Further Reading
More details about clustering in ``SynPlanner`` can be found in <a href="https://synplanner.readthedocs.io/">official documentation</a>.